# web scraping


In [2]:
import requests
from sklearn.utils import shuffle

import json
import pandas as pd
import time
from bs4 import BeautifulSoup

In [3]:
import requests
from bs4 import BeautifulSoup

def scrape_real_estate_data(url,rent=None):
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    print(response.status_code)
    
    
    
    
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract property listings
        listings = soup.find_all('div', class_='lazyload-wrapper')
        
        # Extract data from each listing
        for listing in listings:
            # Extract property details (e.g., price, location, etc.)
            link_tag = listing.find('div', class_='results_card')
            link = link_tag.find('a')['href']
            
            price = listing.find('span', class_='price').text
            if rent:
                _ ,price,_, _ = price.replace("\xa0"," ").split(" ")
            else:
                try:
                    _ ,price = price.strip().replace("\xa0"," ").strip().split(" ")
                    
                except:
                    price = "0,"
                
            title_location_div = listing.find('div', class_='card_title')
            title=title_location_div.find("h2").text.strip()
            location=title_location_div.find("h5").text.strip()
            
            # Extract image URL if available
            image_url_div = listing.find('div', class_='results_card_media')
            image_url = image_url_div.find_all('img')[2]['src'] 

            # desciptions
            desc_tag = listing.find('ul',class_='results_card_details')            
            desc = ""
            
            # Extract the text of the <li> tag and concatenate it with the existing descriptions
            for li_tag in desc_tag.find_all( ['li','p'] ):
                desc +=  li_tag.text + " | "
                
            response_click=requests.get(link)
            if response_click.status_code == 200:
                soup_click = BeautifulSoup(response_click.text,"html.parser")
                sub_listings = soup_click.find("div",class_="tower_description")
                description = sub_listings.find("p").text.strip() if sub_listings.find("p") else ""
            else:
                description=""

                
            # get agents contact
            
            ag_name = listing.find('div',class_='agent_name').find('span').text
            
            ag_whatsapp = listing.find('div',class_='agent_contact').find('a')['href']
            ag_call = listing.find('div',class_='agent_contact').find_all('a')[1]['href']
            
            #get the bed and = bath
            
            bed_bath_div = listing.find("div",class_="about_tour")
            bed , bath , sqft = bed_bath_div.find_all("span")  # this will be 3 list 0-bed 1-bath 2-hight(not using it)
            sqft, _ = sqft.text.split(" ")
            number_of_beds =int(bed.text) if  bed  else 0
            number_of_baths = int(bath.text) if  bath.text.isdigit()  else 0
            size_in_sqft = int(sqft.replace(",","")) if sqft else 0
            
            
            
            properties= {
                'title': title,
                'description': desc,
                'price_in_aed': int(price.replace(",","")), 
                'location': location,
                'number_of_beds': number_of_beds,
                'number_of_baths':number_of_baths ,
                "size_in_sqft": size_in_sqft,
                'sale_type': "rent" if rent else "buy",
                'page_url':link,
                'image_url': image_url,
                'agent_name':ag_name,
                'whatsapp_link':ag_whatsapp,
                'call_agent':ag_call,
                'long_description': description
            }

        return properties
    else:
        print("Failed to fetch the page")



In [ ]:
# ## NOW LETS GRAP SOME MORE PAGES AND STORE TO DF


import numpy as np
from sklearn.utils import shuffle

num_pages = 10

starter = 0
for _ in range(50):
    
    rent=True
    rent_listings= []
    for i in range(num_pages):
        url_rent= f"https://www.allsoppandallsopp.com/dubai/properties/residential/lettings/page-{starter+i+1}"
        listings = scrape_real_estate_data(url_rent,rent)
        rent_listings.append(listings)
#         if (i + 1) % 4 == 0:  # Add a wait every 4 iterations
#             time.sleep(10)  # Adjust the sleep duration
        time.sleep(300)
        print("rent_page : ", f"{starter+i+1}/500")
    rent_df = pd.DataFrame(rent_listings)
    rent_df.reset_index(drop=True, inplace=True)

    time.sleep(300)

    sale_listings= []
    for i in range(num_pages):
        url_sale= f"https://www.allsoppandallsopp.com/dubai/properties/residential/sales/page-{starter+i+1}"
        listings = scrape_real_estate_data(url_sale)
        sale_listings.append(listings)
#         if (i + 1) % 4 == 0:  # Add a wait every 4 iterations
#             time.sleep(10)  # Adjust the sleep duration
        time.sleep(300)
        
        print("sale_page : ", starter+i+1)

        
    sale_df = pd.DataFrame(sale_listings)
    sale_df.reset_index(drop=True, inplace=True)
    

    time.sleep(300)
    starter = starter + 10


rent_json=rent_listings
sale_json=sale_listings

# sale_df_0=sale_df
# rent_df_0=rent_df


rent_df.to_csv("2_real_estate_rents.csv")
sale_df.to_csv("2_real_estate_sales.csv")

sale_final = shuffle(pd.concat([rent_df,sale_df]))
sale_final.to_csv("real_estate_sales.csv")



200
